In [266]:

import pandas as pd
import numpy as np
import math
import glob
import random
from collections import Counter
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag

import spacy
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec
from gensim import corpora


import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hamza.ahmed/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hamza.ahmed/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hamza.ahmed/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hamza.ahmed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/hamza.ahmed/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [267]:

Data = pd.read_csv('/Users/hamza.ahmed/Python Apps/API Access/Youtube_Knife_Reviews.csv')
Data.dropna()

,Channel,Video Title,Video ID,Video Length,Video Description,Video Tags,Video Transcript,Comment
0,Blade HQ,2021 Pocket Knife Holiday Gift Guide! | Knife ...,2_g-SdpHEWY,18m:55s,Need a pocket knife as a gift for your loved o...,"('pocket knife', 'Blade HQ', 'BladeHQ', 'Knife...","How's it going, guys? Welcome to the 2021 gif...",Finally picked up a benchmade bug out after ye...
1,Blade HQ,The Best Kershaw Pocket Knives in 2020 at Blad...,qdbNejPUtUQ,22m:14s,Zac and Jamie sit down at an appropriate dista...,"('Blade HQ', 'KnifeLife', 'KnifeBanter', 'Knif...",today a knife banter we're gonna talk\nkershaw...,Would love to see all the launch’s I would lov...
2,KnifeCenter,Best Automatic Knives - What is a Switchblade?,7FVfAqVx5Is,29m:11s,Automatic knives or switchblades have seen a s...,"('KnifeCenter', 'Knife Center', 'Knifecenter.c...","Hey everyone, David C. Andersen here coming\na...","I have a couple of auto knives, but this video..."
3,KnifeCenter,Best CIVIVI Pocket Knives for EDC of 2020 - Kn...,GEYcxRvLw3g,10m:42s,CIVIVI (the budget subsidiary of We Knife Co) ...,"('KnifeCenter', 'Knife Center', 'Knifecenter.c...",Hey everyone David C Andersen here\ncoming at ...,"I can't believe the elementum is only $50, the..."
4,KnifeCenter,Best Cheap Knives of 2021 - Great EDC Folders ...,995i76Y-Uw0,25m:53s,"When we say cheap knives, we're talking the be...","('KnifeCenter', 'Knife Center', 'Knifecenter.c...","Hey everyone, David C Andersen here coming\nat...",I love this channel. Great audio/lighting qual...
5,KnifeCenter,Best EDC Knives for 5th Pocket Carry of 2021 -...,ZhyJj8XMnPY,17m:51s,"Whether you call it a watch pocket, a coin poc...","('KnifeCenter', 'Knife Center', 'Knifecenter.c...","Hey everyone, David C. Andersen here coming at...",The last year or two I’ve enjoyed carrying the...
6,KnifeCenter,Best Keychain Knives of 2021,54YumGPjO7Q,25m:13s,"Whether for you or as a gift for someone else,...","('KnifeCenter', 'Knife Center', 'Knifecenter.c...","Hey everyone, David C Andersen here coming at\...",The pocket clip on the second knife he shows i...
7,KnifeCenter,"Best Knife from Every Brand in 2021, Part 1",cQOVB5UrOjc,28m:7s,"So many companies make so many great knives, i...","('KnifeCenter', 'Knife Center', 'Knifecenter.c...","Hey everyone, David C Andersen here coming\nat...","David, you are unbelievably good in these vide..."
8,KnifeCenter,Best Pocket Cleaver Folding Knives of 2020 Ava...,8EYNe-Trgqs,12m:4s,Pocket cleavers are some of the most awe-inspi...,"('KnifeCenter', 'Knife Center', 'Knifecenter.c...","Hey, everyone, David C. Andersen here coming\n...",Was honestly never into Pocket cleaves until a...
9,KnifeCenter,Best Pocket Fixed Blade EDC Knives in 2021,5JXBoex_Q9A,23m:40s,Ever wish your KA-BAR was more pocket-friendly...,"('KnifeCenter', 'Knife Center', 'Knifecenter.c...","Hey everyone, David C. Andersen here coming at...",FYI: If you’re going to pocket carry a sheat...


In [274]:
Comment_Data = (comment_words.lower() for comment_words in Data["Comment"])
Tag_Data = (tag_words.lower() for tag_words in Data["Video Tags"])





selected_words=['benchmade', 'crkt', 'kershaw', 'chris reeves', 'civivi', 'weknife']

vect = CountVectorizer(vocabulary=selected_words)
CD = vect.fit_transform(Comment_Data)
CD = CD.toarray()
vectorizer_feature_names = vect.get_feature_names_out()


vect = CountVectorizer(vocabulary=selected_words)
TD = vect.fit_transform(Tag_Data)
TD = TD.toarray()
vectorizer_feature_names = vect.get_feature_names_out()

Knife_Companies2 = pd.DataFrame(data = TD, columns = vectorizer_feature_names)
Knife_Companies2 = Knife_Companies2.rename(columns={col: col+'_tags' for col in Knife_Companies2.columns})
print(Knife_Companies2)




# res = pd.Series(np.ravel((vect.fit_transform(Data['Comment']).sum(axis=0))),
#                          index=vect.get_feature_names_out())

# res



    benchmade_tags  crkt_tags  kershaw_tags  chris reeves_tags  civivi_tags  \
0                0          0             0                  0            0   
1                0          0             1                  0            0   
2                2          0             2                  0            0   
3                0          0             0                  0            9   
4                0          2             1                  0            2   
5                1          0             0                  0            0   
6                0          1             1                  0            1   
7                1          1             1                  0            1   
8                0          0             1                  0            2   
9                1          1             0                  0            0   
10               0          0             3                  0            2   
11               1          2             0         

In [244]:
selected_words=['knife', 'cool', 'awesome']

if any (selected_words in Data["Comment"] for selected_words in selected_words):
    print((selected_words))
    

In [167]:
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(Data["Video Transcript"].values.astype('U'))


In [168]:
doc_term_matrix 

<1825x4657 sparse matrix of type '<class 'numpy.int64'>'
	with 999587 stored elements in Compressed Sparse Row format>

In [170]:
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=5, random_state=42)

In [176]:
for i in range(10):
    random_id = random.randint(0,len(count_vect.get_feature_names()))
    print(count_vect.get_feature_names()[random_id])

uncomfortable
josh
water
initial
chrome
projects
arius
williamson
bolt
file


In [183]:
first_topic = LDA.components_[0]
top_topic_words = first_topic.argsort()[-10:]
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

pocket
tip
definitely
uh
sheath
gonna
carry
fixed
guy
um


In [184]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['pocket', 'tip', 'definitely', 'uh', 'sheath', 'gonna', 'carry', 'fixed', 'guy', 'um']


Top 10 words for topic #1:
['action', 'um', 'particular', 'pocket', 'version', 'design', 'different', 'shape', 'clip', 'lock']


Top 10 words for topic #2:
['military', 'carry', 'need', 'uh', 'look', 'cool', 'mean', 'gonna', 'guys', 'yeah']


Top 10 words for topic #3:
['ll', 'carbon', 'size', 'tang', 'bevel', 'cut', 'let', 'chef', 'survival', 'gonna']


Top 10 words for topic #4:
['automatic', 'lock', 'push', 'action', '2020', 'um', 'guys', 'cool', 'uh', 'yeah']




In [185]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(1825, 5)

In [152]:
# stop = set(stopwords.words('english'))
# exclude = set(string.punctuation)
# lemma = WordNetLemmatizer()


# def clean(doc):
#     stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
#     punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
#     normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
#     return normalized


In [153]:
# doc_clean = [clean(doc).split() for doc in Data["Video Transcript"]]       

In [197]:
x = Data['Video Transcript']
keywords = ('knife', 'Sharpness', 'Sharp')
if keywords in Data['Video Transcript']:
    print (keywords)

0       today on knife banner we are talking the best ...
1       Hey everyone, David C. Andersen here coming\na...
2       hey everybody nakeeran today I got a review fo...
3       i'm will griffin i'm a knife maker the knife i...
4       hey everyone David C Andersen here\ncoming at ...
                              ...                        
1820    hey gang todd helms out here with eastman's \n...
1821    hey gang todd helms out here with eastman's \n...
1822    hey gang todd helms out here with eastman's \n...
1823    hey gang todd helms out here with eastman's \n...
1824    hey gang todd helms out here with eastman's \n...
Name: Video Transcript, Length: 1825, dtype: object

In [125]:

# #pd.DataFrame(zip(Data['Comment'],Data['Comment'].map(lambda d: re.escape("love") in d)))
# dictionary = corpora.Dictionary(doc_clean)
# doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [137]:
# Lda = gensim.models.ldamodel.LdaModel
# ldamodel = Lda(doc_term_matrix, num_topics=50, id2word = dictionary, passes=50)

In [149]:
#print(ldamodel.print_topics(num_topics=5, num_words=50))

In [ ]:
search_list = ['poop']
comm=Data.Comment.str.lower().str.split('\s')

sq=Data.assign(c=comm.apply(lambda x:[*{*x}&{*search_list}])\
.str.join(','),d=comm.apply(lambda \
x:[*{*x}&{*search_list}]).str.len())


# df1 = Data.Comment.str.split().apply(lambda x: Counter(x))
# df1